In [1]:
import logging
import bz2
import sys
import os
from pprint import pprint

In [2]:
import numpy as np

In [3]:
import qcfractal.interface as ptl
import tqdm
import sys
from openff.qcsubmit import workflow_components
from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openforcefield.topology import Molecule

In [4]:
# Need to turn off undefined stereochemistry warnings
logging.getLogger().setLevel(logging.ERROR)

In [5]:
def test_submit(qcs_ds, client):
    result = qcs_ds.submit(client, threads=1)
    print(result)
    print("Total tasks:", sum(result.values()))

def test_submit_to_local_server(qcs_ds):
    print("Submitting to test-run to local server")
    client = ptl.FractalClient("localhost:7777", verify=False)
    test_submit(qcs_ds, client)
    
def test_submit_to_test_server(qcs_ds):
    from qcfractal import FractalSnowflakeHandler
    print("Submitting test-run to in-memory server")
    server = FractalSnowflakeHandler()
    test_submit(qcs_ds, server.client())


In [6]:
sdf_file = "./pubLigsNeutralGoodDensity.sdf"

molecules = Molecule.from_file(sdf_file,
                          "sdf",
                          allow_undefined_stereo=True,
                         )
# Considering molecules that have rotors greater than 3 for fragmentation down the line
mols = []
for molecule in molecules:
    if len(molecule.find_rotatable_bonds()) > 3:
        mols.append(molecule)
    else:
        continue
print("\nFiles loaded; molecules generated.")


Files loaded; molecules generated.


In [7]:
# Generate the workflow to apply to the molecules
qcs_ds = TorsiondriveDatasetFactory()

component = workflow_components.WBOFragmenter()
component.keep_non_rotor_ring_substituents = True
qcs_ds.add_workflow_component(component)

component = workflow_components.EnumerateStereoisomers()
component.max_isomers = 10
component.toolkit = "openeye"
qcs_ds.add_workflow_component(component)

component = workflow_components.StandardConformerGenerator()
component.max_conformers = 10
component.toolkit = "openeye"
component.rms_cutoff = 3
qcs_ds.add_workflow_component(component)

In [8]:
dataset = qcs_ds.create_dataset(dataset_name="Genentech PDB Ligand Expo fragment torsiondrive neutral v1.0",
        molecules=mols,
        description="Genentech PDB Ligand Expo optimization dataset with fragmented neutral molecules",
        tagline="TorsionDriveDataset")
print("Workflow complete; dataset generated.")

WBOFragmenter                 :   0%|                   | 0/519 [00:00<?, ?it/s]

20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARNING fragmenter: Cannot calculate WBO for fragment [H:19][C:4]1([C@@H:3]([C@H:5]([C@@:7]2([C:1](=[O:11])[NH:9][C:2](=[O:12])[NH:10]2)[O:13][C:6]1([H:21])[C:8]([H:22])([H:23])[O:17][H:29])[OH:16])[OH:14])[OH:15]. Continue growing fragment
20:02:51 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:51 WARN

20:02:53 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 3 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:53 WARNING fragmenter: A new stereocenter formed at 

20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:53 WARNING fragmenter: A new stereocenter formed at atom 13 C
20:02:54 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:54 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:54 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:54 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:54 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:54 WARNING fragmenter: A new stereocenter formed at atom 1 C
20:02:54 WARNING fragmenter: Cannot calculate WBO for fragment [H:16][C:3]1([C@H:1]([C@@H:2]([CH2:4][O:8][C:5]1([H:18])[O:

/home/maverick/anaconda3/envs/qcsub_user/lib/python3.7/site-packages/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(13, 'C')], problematic bonds [], SMILES: Cc1cn(c(=O)[nH]c1=O)[C@H]2C[C@@H]([C@@H]3C2C3)O
  problematic_bonds, oechem.OEMolToSmiles(molecule)))


...

20:18:53 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:53 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:53 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:53 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:54 WARNING fragmenter: A new stereocenter formed at atom 24 C
20:18:54 WARNING fragmenter: A new stereocenter formed at atom 24 C
20:18:54 WARNING fragmenter: A new stereocenter formed at atom 24 C
20:18:54 WARNING fragmenter: A new stereocenter formed at atom 24 C
20:18:55 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:55 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:55 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:55 WARNING fragmenter: A new stereocenter formed at atom 14 C
20:18:55 WARNING fragmenter: A new stereocenter formed at atom 19 N


/home/maverick/anaconda3/envs/qcsub_user/lib/python3.7/site-packages/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(14, 'N')], problematic bonds [], SMILES: [H:26][C:7]1([CH2:9][C:8]([C:10]1([H:31])[N:19]([c:4]2[c:2]3[cH:5][cH:6][nH:17][c:3]3[n:15][cH:1][n:16]2)[C:12]([H:35])([H:36])[H:37])([H:28])[H:29])[H:27]
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/home/maverick/anaconda3/envs/qcsub_user/lib/python3.7/site-packages/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(18, 'N')], problematic bonds [], SMILES: CCCS(=O)(=O)NC1CC(C1)N(C)c2c3cc[nH]c3ncn2
  problematic_bonds, oechem.OEMolToSmiles(molecule)))


20:18:58 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:18:58 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:18:58 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:18:58 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:01 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:01 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:01 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:01 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:06 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:06 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:06 WARNING fragmenter: A new stereocenter formed at atom 21 C
20:19:06 WARNING fragmenter: A new stereocenter formed at atom 21 C


Preparation                   : 100%|███████| 1448/1448 [00:51<00:00, 27.87it/s]

Workflow complete; dataset generated.


In [9]:
dataset.metadata.short_description = "TorsionDrive set"
dataset.metadata.long_description_url = "https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-02-22-Genentech-PDB-Ligand-Expo-fragment-torsiondrive-neutral-v1.0"
dataset.metadata.long_description = "Genentech PDB Ligand Expo neutral molecules set"

confs = np.array([len(mol.conformers) for mol in dataset.molecules])
print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max", 
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())
dataset.metadata.submitter = "pavankum"
dataset.export_dataset("dataset.json.bz2")

Number of unique molecules        1448
Number of filtered molecules      98
Number of conformers              3816
Number of conformers min mean max 1   1.02 3


In [10]:
pprint(dataset.metadata.dict())

{'collection_type': 'TorsiondriveDataset',
 'creation_date': datetime.date(2021, 3, 24),
 'dataset_name': 'Genentech PDB Ligand Expo fragment torsiondrive neutral v1.0',
 'elements': {'N', 'F', 'S', 'Cl', 'H', 'O', 'Br', 'I', 'C'},
 'long_description': 'Genentech PDB Ligand Expo neutral molecules set',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-02-22-Genentech-PDB-Ligand-Expo-fragment-torsiondrive-neutral-v1.0', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-02-22-Genentech-PDB-Ligand-Expo-fragment-torsiondrive-neutral-v1.0'),
 'short_description': 'TorsionDrive set',
 'submitter': 'pavankum'}


In [11]:
%time test_submit_to_test_server(dataset)

Submitting test-run to in-memory server
{'default': 3816}
Total tasks: 3816
CPU times: user 12min 33s, sys: 7.02 s, total: 12min 40s
Wall time: 31min 53s


In [12]:
from openff.qcsubmit.datasets import TorsiondriveDataset
dataset = TorsiondriveDataset.parse_file('./dataset.json.bz2')
dataset.metadata.long_description = "This is the third of the Genentech PDB Ligand Expo Datasets referenced here (https://github.com/openforcefield/qca-dataset-submission/pull/48). This contains torsiondrives for fragmented neutral molecules. Conformers were generated using a RMS cutoff of 3 Ångstroms."
dataset.metadata.submitter = 'pavankum'
dataset.export_dataset("dataset.json.bz2")

In [13]:
dataset.visualize('visualize.pdf')

In [14]:
for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    pprint(obj.dict())

Spec: default
{'basis': 'DZVP',
 'implicit_solvent': None,
 'method': 'B3LYP-D3BJ',
 'program': 'psi4',
 'spec_description': 'Standard OpenFF optimization quantum chemistry '
                     'specification.',
 'spec_name': 'default',
 'store_wavefunction': 'none'}
